In [1]:
import pandas as pd
from transformers import  AlbertConfig, AlbertModel
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
from transformers import AlbertTokenizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Input, Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import numpy as np

/home/neil/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-09-13 21:05:07.257725: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-13 21:05:08.795730: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
Dataset_file = "TV.csv"
df = pd.read_csv(Dataset_file)
label_encoder = LabelEncoder()
df['label_encoded'] = label_encoder.fit_transform(df['Cảm xúc'])

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)



In [3]:
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')

def tokenize_text(text):
    return tokenizer.encode(text, add_special_tokens=True, max_length=128, padding='max_length', truncation=True)


In [4]:
train_df['input_ids'] = train_df['Nhận xét đánh giá'].apply(tokenize_text)
val_df['input_ids'] = val_df['Nhận xét đánh giá'].apply(tokenize_text)



In [5]:
albert_config = AlbertConfig(
    vocab_size=30000,            # Size of the vocabulary
    hidden_size=512,            # Size of the hidden layers
    num_hidden_layers=8,        # Number of hidden layers
    num_attention_heads=10,      # Number of attention heads
    intermediate_size=1024,     # Size of the intermediate (feed-forward) layers
    hidden_dropout_prob=0.2,    # Dropout probability for hidden layers
    attention_probs_dropout_prob=0.2,  # Dropout probability for attention scores
    max_position_embeddings=128,  # Maximum position embeddings (adjust based on your sequence length)
    type_vocab_size=2,          # Number of token types (typically 0 for regular text, 1 for special tokens)
)



In [8]:
input_layer = Input(shape=(128,), dtype=tf.int32)
embedding_layer = Embedding(input_dim=albert_config.vocab_size, output_dim=albert_config.hidden_size)(input_layer)
pooling_layer = GlobalAveragePooling1D()(embedding_layer)
output_layer = Dense(units=len(label_encoder.classes_), activation='softmax')(pooling_layer)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer=Adam(learning_rate=1e-4), loss=SparseCategoricalCrossentropy(from_logits=False), metrics=['accuracy'])


2023-09-13 21:10:38.804164: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 61440000 exceeds 10% of free system memory.
2023-09-13 21:10:38.864518: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 61440000 exceeds 10% of free system memory.


In [7]:
train_input_ids = np.array(train_df['input_ids'].to_list())


NameError: name 'test_df' is not defined

In [ ]:
history = model.fit(
    train_input_ids, train_df['label_encoded'].values,
    validation_data=(val_input_ids, val_df['label_encoded'].values),
    epochs=10, batch_size=32) 

Epoch 1/10
415/415 [==============================] - 74s 156ms/step - loss: 1.5593 - accuracy: 0.4118 - val_loss: 1.3704 - val_accuracy: 0.4337
Epoch 2/10
415/415 [==============================] - 63s 152ms/step - loss: 1.2796 - accuracy: 0.4376 - val_loss: 1.2464 - val_accuracy: 0.4337
Epoch 3/10
415/415 [==============================] - 62s 149ms/step - loss: 1.2200 - accuracy: 0.4376 - val_loss: 1.2246 - val_accuracy: 0.4337
Epoch 4/10
415/415 [==============================] - 62s 149ms/step - loss: 1.2037 - accuracy: 0.4376 - val_loss: 1.2134 - val_accuracy: 0.4334
Epoch 5/10
415/415 [==============================] - 62s 149ms/step - loss: 1.1922 - accuracy: 0.4375 - val_loss: 1.2028 - val_accuracy: 0.4331
Epoch 6/10
415/415 [==============================] - 62s 149ms/step - loss: 1.1808 - accuracy: 0.4382 - val_loss: 1.1912 - val_accuracy: 0.4334
Epoch 7/10
415/415 [==============================] - 62s 149ms/step - loss: 1.1687 - accuracy: 0.4403 - val_loss: 1.1784 - val_ac

In [ ]:
test_input_ids = np.array(test_df['input_ids'].to_list())
test_loss, test_accuracy = model.evaluate(test_input_ids, test_df['label_encoded'].values)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")


104/104 [==============================] - 0s 3ms/step - loss: 1.1373 - accuracy: 0.4741
Test Loss: 1.1373, Test Accuracy: 0.4741


test_loss, test_accuracy = model.evaluate(test_df['input_ids'].to_list(), test_df['label_encoded'].values)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

In [ ]:
model.save('sentiment_model')

INFO:tensorflow:Assets written to: sentiment_model/assets


INFO:tensorflow:Assets written to: sentiment_model/assets


In [ ]:
sample_text = ""
encoded_text = tokenize_text(sample_text)
encoded_text = np.array([encoded_text]) 
predicted_class = model.predict(encoded_text)[0]
predicted_sentiment = label_encoder.inverse_transform([predicted_class.argmax()])[0]
print(f"Predicted Sentiment: {predicted_sentiment}")

1/1 [==============================] - 0s 102ms/step
Predicted Sentiment: tốt
